In [5]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import random

def next_batch(num_split, feature, labels):
    ind = np.arange(0, len(feature))
    np.random.shuffle(ind)
    ind = ind[:num_split]
    feature_shuffle = [feature[i].flatten() for i in ind]
    labels_shuffle = [labels[i] for i in ind]

    return np.asarray(feature_shuffle), np.asarray(labels_shuffle)

x_train = np.load("../saved_data/x_train.npy", allow_pickle=True)
x_test = np.load("../saved_data/x_test.npy", allow_pickle=True)
y_train = np.load("../saved_data/y_train.npy", allow_pickle=True)
y_test = np.load("../saved_data/y_test.npy", allow_pickle=True)

x_train = x_train/255.
x_test = x_test/255.

# Parameters
learning_rate = 0.0005
training_epochs = 100
batch_size = 256
display_step = 1

x = tf.placeholder(tf.float32, [None, 22500])
y = tf.placeholder(tf.float32, [None, 2])

W1 = tf.Variable(tf.random_normal([22500, 256], stddev=0.03), name='W1')
b1 = tf.Variable(tf.random_normal([256]), name='b1')
z1 = tf.nn.relu(tf.matmul(x,W1)+b1)

W2 = tf.Variable(tf.random_normal([256, 128], stddev=0.03), name='W2')
b2 = tf.Variable(tf.random_normal([128]), name='b2')
z2 = tf.nn.relu(tf.matmul(z1,W2)+b2)

W3 = tf.Variable(tf.random_normal([128, 64], stddev=0.03), name='W3')
b3 = tf.Variable(tf.random_normal([64]), name='b3')
z3 = tf.nn.relu(tf.matmul(z2,W3)+b3)

W4 = tf.Variable(tf.random_normal([64, 2], stddev=0.03), name='W4')
b4 = tf.Variable(tf.random_normal([2]), name='b4')


log = tf.matmul(z3, W4) + b4
pred = tf.nn.softmax(tf.matmul(z3, W4) + b4)


correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=log))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)
#optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(len(x_train)/batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = next_batch(batch_size, x_train, y_train)
            _, c, acc = sess.run([optimizer, cost, accuracy], feed_dict={x: batch_xs, y: batch_ys})
            avg_cost += c / total_batch
        if (epoch+1) % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost), "acc=", "{:.5}".format(acc))
    print("Optimization Finished!")
    
    x_test_flat = []
    for i in x_test:
        x_test_flat.append(i.flatten())
    x_test_flat = np.array(x_test_flat)
    
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    print("Accuracy:", accuracy.eval({x: x_test_flat, y: y_test}))

Epoch: 0001 cost= 0.800442553 acc= 0.46094
Epoch: 0002 cost= 0.661687458 acc= 0.67188
Epoch: 0003 cost= 0.597728425 acc= 0.79688
Epoch: 0004 cost= 0.498753360 acc= 0.85547
Epoch: 0005 cost= 0.352910599 acc= 0.90234
Epoch: 0006 cost= 0.310622749 acc= 0.88281
Epoch: 0007 cost= 0.264587562 acc= 0.90234
Epoch: 0008 cost= 0.222283897 acc= 0.89844
Epoch: 0009 cost= 0.205900557 acc= 0.92578
Epoch: 0010 cost= 0.172060169 acc= 0.89844
Epoch: 0011 cost= 0.152673110 acc= 0.94531
Epoch: 0012 cost= 0.145141868 acc= 0.94141
Epoch: 0013 cost= 0.144996414 acc= 0.95703
Epoch: 0014 cost= 0.128586856 acc= 0.91797
Epoch: 0015 cost= 0.147968813 acc= 0.91016
Epoch: 0016 cost= 0.128446989 acc= 0.94922
Epoch: 0017 cost= 0.128743073 acc= 0.91797
Epoch: 0018 cost= 0.118879227 acc= 0.9375
Epoch: 0019 cost= 0.115028999 acc= 0.93359
Epoch: 0020 cost= 0.114000840 acc= 0.96094
Epoch: 0021 cost= 0.145389184 acc= 0.96484
Epoch: 0022 cost= 0.123604736 acc= 0.94531
Epoch: 0023 cost= 0.111817353 acc= 0.97266
Epoch: 0024 